In [2]:
import tensorflow as tf  # 引入tensorflow库
from tensorflow.examples.tutorials.mnist import input_data  # 载入minist数据集
import numpy as np

mnist = input_data.read_data_sets(
    "MNIST_data/", one_hot=True, source_url="http://storage.googleapis.com/cvdf-datasets/mnist/")

images_train = mnist.train.images
images_test = mnist.test.images
labels_train = mnist.train.labels
labels_test = mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
n_inputs = 28 * 28  # MNIST 输入层维度
n_hidden1 = 300  # 第一层隐藏层神经元数目
n_hidden2=100  # 第二层隐藏层神经元数目
n_outputs=10  # 输出层神经元数目 = 分类的数目

In [23]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
y = tf.placeholder(tf.float32, shape=[None, 10], name="y")

In [24]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.variable_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal_initializer(stddev=stddev)
        w = tf.get_variable(
            name="weights", shape=(n_inputs, n_neurons), initializer=init, dtype='float32')
        b = tf.get_variable(
            name="biases", shape=[n_neurons], initializer=tf.constant_initializer(0), dtype='float32')
        z = tf.matmul(X, w) + b
        activation = activation or tf.nn.relu
        return activation(z)


In [25]:
with tf.variable_scope('dnn'):
    hidden1 = neuron_layer(x, n_hidden1, "hidden1", activation=tf.nn.elu)
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation=tf.nn.elu)
    logits = neuron_layer(hidden2, n_outputs, "outputs", activation=tf.identity)

In [26]:
with tf.name_scope('loss'):
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(entropy, name="loss")

In [27]:
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [28]:
with tf.name_scope("eval"):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(logits, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [29]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [30]:
with tf.name_scope("log"):
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)
    file_writer = tf.summary.FileWriter("2-2_network", tf.get_default_graph())

In [32]:
batch_size=50
n_epochs = 400

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(images_train.shape[0] // batch_size):
            start = iteration * batch_size
            x_batch = images_train[start: start + batch_size]
            y_batch = labels_train[start: start + batch_size]
            sess.run(training_op, feed_dict={x: x_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={x: x_batch, y: y_batch})
        acc_test, accuracy_summary_str = sess.run(
            [accuracy, accuracy_summary],
            feed_dict={x: images_test, y: labels_test}
        )
        file_writer.add_summary(accuracy_summary_str, epoch * images_train.shape[0] // batch_size)
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

    save_path=saver.save(sess, "./final_model.ckpt")

0 Train accuracy: 0.98 Test accuracy: 0.9078
1 Train accuracy: 0.98 Test accuracy: 0.9211
2 Train accuracy: 1.0 Test accuracy: 0.9302
3 Train accuracy: 1.0 Test accuracy: 0.9388
4 Train accuracy: 1.0 Test accuracy: 0.9441
5 Train accuracy: 1.0 Test accuracy: 0.9479
6 Train accuracy: 1.0 Test accuracy: 0.9529
7 Train accuracy: 1.0 Test accuracy: 0.9554
8 Train accuracy: 1.0 Test accuracy: 0.9592
9 Train accuracy: 1.0 Test accuracy: 0.9614
10 Train accuracy: 1.0 Test accuracy: 0.9625
11 Train accuracy: 1.0 Test accuracy: 0.964
12 Train accuracy: 1.0 Test accuracy: 0.965
13 Train accuracy: 1.0 Test accuracy: 0.966
14 Train accuracy: 1.0 Test accuracy: 0.9671
15 Train accuracy: 1.0 Test accuracy: 0.9674
16 Train accuracy: 1.0 Test accuracy: 0.9674
17 Train accuracy: 1.0 Test accuracy: 0.9677
18 Train accuracy: 1.0 Test accuracy: 0.968
19 Train accuracy: 1.0 Test accuracy: 0.9685
20 Train accuracy: 1.0 Test accuracy: 0.9684
21 Train accuracy: 1.0 Test accuracy: 0.9689
22 Train accuracy: 1.0

181 Train accuracy: 1.0 Test accuracy: 0.9791
182 Train accuracy: 1.0 Test accuracy: 0.9791
183 Train accuracy: 1.0 Test accuracy: 0.9791
184 Train accuracy: 1.0 Test accuracy: 0.9791
185 Train accuracy: 1.0 Test accuracy: 0.979
186 Train accuracy: 1.0 Test accuracy: 0.979
187 Train accuracy: 1.0 Test accuracy: 0.979
188 Train accuracy: 1.0 Test accuracy: 0.979
189 Train accuracy: 1.0 Test accuracy: 0.979
190 Train accuracy: 1.0 Test accuracy: 0.979
191 Train accuracy: 1.0 Test accuracy: 0.9789
192 Train accuracy: 1.0 Test accuracy: 0.979
193 Train accuracy: 1.0 Test accuracy: 0.9788
194 Train accuracy: 1.0 Test accuracy: 0.9787
195 Train accuracy: 1.0 Test accuracy: 0.9787
196 Train accuracy: 1.0 Test accuracy: 0.9787
197 Train accuracy: 1.0 Test accuracy: 0.9787
198 Train accuracy: 1.0 Test accuracy: 0.9787
199 Train accuracy: 1.0 Test accuracy: 0.9787
200 Train accuracy: 1.0 Test accuracy: 0.9787
201 Train accuracy: 1.0 Test accuracy: 0.9787
202 Train accuracy: 1.0 Test accuracy: 0.

361 Train accuracy: 1.0 Test accuracy: 0.9792
362 Train accuracy: 1.0 Test accuracy: 0.9792
363 Train accuracy: 1.0 Test accuracy: 0.9792
364 Train accuracy: 1.0 Test accuracy: 0.9792
365 Train accuracy: 1.0 Test accuracy: 0.9792
366 Train accuracy: 1.0 Test accuracy: 0.9792
367 Train accuracy: 1.0 Test accuracy: 0.9792
368 Train accuracy: 1.0 Test accuracy: 0.9792
369 Train accuracy: 1.0 Test accuracy: 0.9792
370 Train accuracy: 1.0 Test accuracy: 0.9792
371 Train accuracy: 1.0 Test accuracy: 0.9792
372 Train accuracy: 1.0 Test accuracy: 0.9792
373 Train accuracy: 1.0 Test accuracy: 0.9793
374 Train accuracy: 1.0 Test accuracy: 0.9793
375 Train accuracy: 1.0 Test accuracy: 0.9793
376 Train accuracy: 1.0 Test accuracy: 0.9793
377 Train accuracy: 1.0 Test accuracy: 0.9793
378 Train accuracy: 1.0 Test accuracy: 0.9793
379 Train accuracy: 1.0 Test accuracy: 0.9793
380 Train accuracy: 1.0 Test accuracy: 0.9793
381 Train accuracy: 1.0 Test accuracy: 0.9793
382 Train accuracy: 1.0 Test accur